# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "City_Data.csv"
file_df = pd.read_csv(file)
file_df.head()
clean_file_df = file_df.dropna()
clean_file_df.head()

,City,City_ID,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind_speed,Country,Date
0,Gazimurskiy Zavod,2024122,51.55,118.33,2.19,85,100,4.18,RU,2020-01-30 02:08:53
1,Rio Grande,3451138,-32.03,-52.10,75.99,78,73,5.73,BR,2020-01-30 02:08:53
2,Airai,1651810,-8.93,125.41,83.88,59,33,3.33,TL,2020-01-30 02:05:58
3,Punta Arenas,3874787,-53.15,-70.92,50.00,76,75,25.28,CL,2020-01-30 02:06:38
4,San Cristobal,3628473,7.77,-72.22,80.60,74,40,3.36,VE,2020-01-30 02:07:55


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
location = clean_file_df[["Latitude", "Longitude"]]
humidity = clean_file_df["Humidity"].astype(float)


In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, max_intensity = 100,
                                dissipating=False, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Vacation criteria:
# Criteria_1: Max temperature between 70 and 85 degrees Farenheit
# Criteria_2: Humidity less than 50%

ideal_vac_df = clean_file_df[(clean_file_df["Temperature (F)"] > 70) & (clean_file_df["Temperature (F)"] < 85)
                             & (clean_file_df["Humidity"] < 50) ]
ideal_vac_df.dropna()


,City,City_ID,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind_speed,Country,Date
14,Bengkulu,1649150,-3.80,102.27,84.54,47,100,5.46,ID,2020-01-30 02:05:01
15,Yaan,2338660,7.38,8.57,70.92,14,7,4.36,NG,2020-01-30 02:08:57
46,Kaitangata,2208248,-46.28,169.85,71.01,42,100,17.00,NZ,2020-01-30 02:09:26
54,Codrington,2171099,-38.27,141.97,76.53,44,0,8.48,AU,2020-01-30 02:09:28
117,Puerto Ayora,3652764,-0.74,-90.35,77.00,27,52,1.01,EC,2020-01-30 02:09:42
141,Basoko,219414,1.24,23.62,74.61,49,60,2.57,CD,2020-01-30 02:09:48
231,Orocue,3673536,4.79,-71.34,82.98,47,16,5.64,CO,2020-01-30 02:10:11
354,Nioro,2412408,13.35,-15.75,70.88,10,59,3.24,GM,2020-01-30 02:10:47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Mapping hotel
hotel_df = ideal_vac_df.rename(columns={"Latitude": "Lat",
                                        "Longitude": "Lng"})
hotel_df["Hotel Name"] = ""
hotel_df.reset_index()

# Search parameters
params = {
        "types": "lodging",
        "radius": 5000,
        "key": g_key
        }

In [13]:
# Get Google Places API for ideal vacation spots
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]
    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    # print(response.url) - # Uncomment to review response.  Comment it back.  DON'T EXPOSE KEY!

    my_hotels = response.json()
    # print(json.dumps(my_hotels, indent=4, sort_keys = True)) - UNCOMMENT if you want to print json format output.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = my_hotels["results"][0]["name"]
               
    except (NameError, KeyError, IndexError):
        print("Hotel not found")

hotel_df

Hotel not found
Hotel not found


,City,City_ID,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind_speed,Country,Date,Hotel Name
14,Bengkulu,1649150,-3.80,102.27,84.54,47,100,5.46,ID,2020-01-30 02:05:01,Grage Hotel Bengkulu
15,Yaan,2338660,7.38,8.57,70.92,14,7,4.36,NG,2020-01-30 02:08:57,
46,Kaitangata,2208248,-46.28,169.85,71.01,42,100,17.00,NZ,2020-01-30 02:09:26,Kaitangata Motor Camp
54,Codrington,2171099,-38.27,141.97,76.53,44,0,8.48,AU,2020-01-30 02:09:28,Codrington Gardens Bed And Breakfast
117,Puerto Ayora,3652764,-0.74,-90.35,77.00,27,52,1.01,EC,2020-01-30 02:09:42,Finch Bay Galapagos Hotel
141,Basoko,219414,1.24,23.62,74.61,49,60,2.57,CD,2020-01-30 02:09:48,Flat Hotel La Benediction
231,Orocue,3673536,4.79,-71.34,82.98,47,16,5.64,CO,2020-01-30 02:10:11,Hotel Brisas de San Miguel
354,Nioro,2412408,13.35,-15.75,70.88,10,59,3.24,GM,2020-01-30 02:10:47,


In [14]:
narrowed_city_df = hotel_df
narrowed_city_df.head(10)

,City,City_ID,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind_speed,Country,Date,Hotel Name
14,Bengkulu,1649150,-3.80,102.27,84.54,47,100,5.46,ID,2020-01-30 02:05:01,Grage Hotel Bengkulu
15,Yaan,2338660,7.38,8.57,70.92,14,7,4.36,NG,2020-01-30 02:08:57,
46,Kaitangata,2208248,-46.28,169.85,71.01,42,100,17.00,NZ,2020-01-30 02:09:26,Kaitangata Motor Camp
54,Codrington,2171099,-38.27,141.97,76.53,44,0,8.48,AU,2020-01-30 02:09:28,Codrington Gardens Bed And Breakfast
117,Puerto Ayora,3652764,-0.74,-90.35,77.00,27,52,1.01,EC,2020-01-30 02:09:42,Finch Bay Galapagos Hotel
141,Basoko,219414,1.24,23.62,74.61,49,60,2.57,CD,2020-01-30 02:09:48,Flat Hotel La Benediction
231,Orocue,3673536,4.79,-71.34,82.98,47,16,5.64,CO,2020-01-30 02:10:11,Hotel Brisas de San Miguel
354,Nioro,2412408,13.35,-15.75,70.88,10,59,3.24,GM,2020-01-30 02:10:47,


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
# Display Map

gmaps.configure(api_key=g_key)
location = clean_file_df[["Latitude", "Longitude"]]
humidity = clean_file_df["Humidity"].astype(float)
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_layer = gmaps.heatmap_layer(location, weights=humidity, max_intensity = 100,
                                dissipating=False, point_radius=2)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))